In [24]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder,StandardScaler

from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

In [25]:
df = pd.read_csv('cleaned_data.csv')
df.head()

,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,...,day_of_week,is_month_start,is_month_end,is_quarter_start,is_quarter_end,is_year_start,is_year_end,is_weekend,order_prepare_time,distance
0,37.0,4.9,22.745049,75.892471,22.765049,75.912471,Sunny,High,2,Snack,...,5,0,0,0,0,0,0,1,15.0,3
1,34.0,4.5,12.913041,77.683237,13.043041,77.813237,Stormy,Jam,2,Snack,...,4,0,0,0,0,0,0,0,5.0,20
2,23.0,4.4,12.914264,77.678400,12.924264,77.688400,Sandstorms,Low,0,Drinks,...,5,0,0,0,0,0,0,1,15.0,1
3,38.0,4.7,11.003669,76.976494,11.053669,77.026494,Sunny,Medium,0,Buffet,...,1,0,0,0,0,0,0,0,10.0,7
4,32.0,4.6,12.972793,80.249982,13.012793,80.289982,Cloudy,High,1,Snack,...,5,0,0,0,0,0,0,1,15.0,6


In [26]:
df.columns

Index(['Delivery_person_Age', 'Delivery_person_Ratings', 'Restaurant_latitude',
       'Restaurant_longitude', 'Delivery_location_latitude',
       'Delivery_location_longitude', 'Weather_conditions',
       'Road_traffic_density', 'Vehicle_condition', 'Type_of_order',
       'Type_of_vehicle', 'multiple_deliveries', 'Festival', 'City',
       'Time_taken(min)', 'City_code', 'day', 'month', 'quarter', 'year',
       'day_of_week', 'is_month_start', 'is_month_end', 'is_quarter_start',
       'is_quarter_end', 'is_year_start', 'is_year_end', 'is_weekend',
       'order_prepare_time', 'distance'],
      dtype='object')

In [27]:
#df.replace(["NaN","nan","Nan","None",""],np.nan,regex=True,inplace=True)

In [28]:
df.drop(columns=['City_code'],axis=1,inplace=True)

In [29]:
X = df.drop(columns=['Time_taken(min)'],axis=1)
y = df['Time_taken(min)']

In [30]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [31]:
X_train.shape

(36474, 28)

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45593 entries, 0 to 45592
Data columns (total 29 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Delivery_person_Age          45593 non-null  float64
 1   Delivery_person_Ratings      45593 non-null  float64
 2   Restaurant_latitude          45593 non-null  float64
 3   Restaurant_longitude         45593 non-null  float64
 4   Delivery_location_latitude   45593 non-null  float64
 5   Delivery_location_longitude  45593 non-null  float64
 6   Weather_conditions           45593 non-null  object 
 7   Road_traffic_density         45593 non-null  object 
 8   Vehicle_condition            45593 non-null  int64  
 9   Type_of_order                45593 non-null  object 
 10  Type_of_vehicle              45593 non-null  object 
 11  multiple_deliveries          45593 non-null  float64
 12  Festival                     45593 non-null  object 
 13  City            

#### OneHotEncoding

In [33]:
ohe = OneHotEncoder()

In [34]:
cat_col = X.select_dtypes(include=['object']).columns
num_col = X.select_dtypes(include=['float64','int64']).columns

### Feature Encoding

In [35]:
# Fit and transform the categorical columns
X_train[cat_col] = X_train[cat_col].apply(lambda x: x.str.strip())
ohe.fit(X_train[cat_col])
X_train_cat_encoded = ohe.transform(X_train[cat_col])

# Create a dataframe from the encoded categorical data
X_train_encoded_df = pd.DataFrame(X_train_cat_encoded.toarray(), columns=ohe.get_feature_names_out(cat_col))

# Reset the index of both the original and encoded dataframes
X_train_encoded_df = X_train_encoded_df.reset_index(drop=True)
X_train = X_train.reset_index(drop=True)

# Drop the original categorical columns
X_train.drop(columns=cat_col, axis=1, inplace=True)

# Concatenate the original data with the encoded data
X_train_final = pd.concat([X_train, X_train_encoded_df], axis=1)

# Now X_train_final should have no NaN values, and the row count should be correct.


In [36]:
ohe.get_feature_names_out()

array(['Weather_conditions_Cloudy', 'Weather_conditions_Fog',
       'Weather_conditions_Sandstorms', 'Weather_conditions_Stormy',
       'Weather_conditions_Sunny', 'Weather_conditions_Windy',
       'Road_traffic_density_High', 'Road_traffic_density_Jam',
       'Road_traffic_density_Low', 'Road_traffic_density_Medium',
       'Type_of_order_Buffet', 'Type_of_order_Drinks',
       'Type_of_order_Meal', 'Type_of_order_Snack',
       'Type_of_vehicle_bicycle', 'Type_of_vehicle_electric_scooter',
       'Type_of_vehicle_motorcycle', 'Type_of_vehicle_scooter',
       'Festival_No', 'Festival_Yes', 'City_Metropolitian',
       'City_Semi-Urban', 'City_Urban'], dtype=object)

In [37]:
# Transform the categorical columns in X_test using the already fitted encoder
X_test[cat_col] = X_test[cat_col].apply(lambda x: x.str.strip())
X_test_cat_encoded = ohe.transform(X_test[cat_col])

# Create a dataframe from the encoded categorical data for X_test
X_test_encoded_df = pd.DataFrame(X_test_cat_encoded.toarray(), columns=ohe.get_feature_names_out(cat_col))

# Reset the index of both the original and encoded dataframes for X_test
X_test_encoded_df = X_test_encoded_df.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

# Drop the original categorical columns from X_test
X_test.drop(columns=cat_col, axis=1, inplace=True)

# Concatenate the original data with the encoded data for X_test
X_test_final = pd.concat([X_test, X_test_encoded_df], axis=1)

# Now X_test_final should have no NaN values, and the row count should be correct.


In [38]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.model_selection import cross_val_score

In [17]:
lr = LinearRegression()
lr.fit(X_train_final,y_train)

LinearRegression()

In [18]:
y_pred = lr.predict(X_test_final)

In [19]:
r2_score(y_test,y_pred)

0.5490941559690284

In [20]:
dectree = DecisionTreeRegressor(max_depth=7)
dectree.fit(X_train_final,y_train)
y_pred = dectree.predict(X_test_final)
score = r2_score(y_test,y_pred)
score

0.689757998821662

In [39]:
model = RandomForestRegressor()
model.fit(X_train_final,y_train)
y_pred = model.predict(X_test_final)
score = r2_score(y_test,y_pred)
score

0.8125234970455169

In [63]:
mse = mean_squared_error(y_test,y_pred)
mse

np.float64(39.584544438917476)

In [33]:
print(np.mean(cross_val_score(model,X_train_final,y_train,cv=5,scoring='r2')))
print(np.mean(cross_val_score(model,X_test_final,y_test,cv=5,scoring='r2')))

0.8102249658077845
0.7940192441868873


## Feature Selection

In [43]:
from mlxtend.feature_selection import SequentialFeatureSelector  as SFS
from sklearn.preprocessing import LabelEncoder

In [45]:
categorical_columns = df.select_dtypes(include='object').columns
label_encoder = LabelEncoder()
df[categorical_columns] = df[categorical_columns].apply(lambda col: label_encoder.fit_transform(col))

In [47]:
X = df.drop('Time_taken(min)', axis=1)  # Features
y = df['Time_taken(min)']  # Target variable

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [48]:
model = RandomForestRegressor()

sfs = SFS(model, 
          k_features=25,  # Number of features to select
          forward=True,   # Forward selection, False for backward
          floating=False, # If True, it dynamically adds/removes features
          scoring='accuracy', 
          cv=5)

sfs = sfs.fit(X_train,y_train)


STOPPING EARLY DUE TO KEYBOARD INTERRUPT...